# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from fasthtml.common import *
from monsterui.all import *
from memexplatform_obsidian.commons import config, MountPaths, ResponseTypes, ExtensionTypes
from memexplatform.ui.structure import ifhtmx
from memexplatform_obsidian.mdmanager import ObsidianHTMLRenderer, get_subdirs, resolve_note_path
from mistletoe import Document
import urllib
from typing import Optional
import pathlib
from pathlib import Path
from memexplatform_obsidian.mdmanager import ObsidianPage, guess_mime
from memexplatform_obsidian.datastore import DBStore, FileStore
from io import BytesIO
import uuid
from memexplatform_obsidian.home import homepage

In [ ]:
# Theme.neutral.headers?

In [ ]:
#| export
def create_app():
    hdrs = Theme.neutral.headers(highlightjs=True, apex_charts=True, katex=True)
    Mermaid = MermaidJS()
    Katex = KatexMarkdownJS()
    D3 = Script(src="https://cdn.jsdelivr.net/npm/d3@7")
    PyscriptCSS=Link(rel="stylesheet", href="https://pyscript.net/releases/2025.8.1/core.css", type="text/css"),
    PyscriptJS=Script(src="https://pyscript.net/releases/2025.8.1/core.js", type='module')
    SurrealGlobalAdd = Script("surreal.globalsAdd());");
    Plotly = Script(src="https://cdn.plot.ly/plotly-3.0.1.min.js")
     # Observable (OJS) runtime + stdlib
    # ObservableJS runtime with HTMX support
    OJSRuntime = Script(type="module", content="""
import {Runtime, Inspector} from "https://cdn.jsdelivr.net/npm/@observablehq/runtime@5/dist/runtime.js";
import {Library} from "https://cdn.jsdelivr.net/npm/@observablehq/stdlib@5/dist/stdlib.js";
import {parser} from "https://cdn.jsdelivr.net/npm/@observablehq/parser@5/dist/parser.min.js";

// Wait for DOM to be ready
document.addEventListener('DOMContentLoaded', function() {
    // Find all <script type="ojs"> and process them
    document.querySelectorAll("script[type='ojs']").forEach(script => {
        const containerId = script.dataset.container;
        const container = document.getElementById(containerId);
        const code = script.textContent;
        
        if (!container) {
            console.error('Container not found:', containerId);
            return;
        }
        
        try {
            // Parse OJS source into cells
            const parsed = parser.parseModule(code);
            const runtime = new Runtime(new Library());
            const main = runtime.module(parsed);
            
            // Create inspector for the container
            const inspector = new Inspector(container);
            
            // For each cell in the module, create a variable
            for (const cell of parsed.cells) {
                if (cell.id && cell.id.name) {
                    main.variable(inspector).define(cell.id.name, cell.body);
                } else {
                    // Anonymous cells
                    main.variable(inspector).define(null, cell.body);
                }
            }
        } catch (error) {
            console.error('Error parsing OJS code:', error);
            container.innerHTML = `<div style="color: red;">Error rendering OJS: ${error.message}</div>`;
        }
    });
});
""")
    

    GradioLiteCSS = Link(rel="stylesheet", href="https://cdn.jsdelivr.net/npm/@gradio/lite/dist/lite.css")
    GradioLiteJS = Script(type="module", crossorigin=True, src="https://cdn.jsdelivr.net/npm/@gradio/lite/dist/lite.js")
    add_hdrs = [Mermaid, Katex, D3, PyscriptCSS, PyscriptJS, SurrealGlobalAdd, Plotly, OJSRuntime, GradioLiteCSS, GradioLiteJS]
    newhdrs = hdrs +add_hdrs
    mount_routes = []

    app = FastHTML(hdrs=newhdrs, live=True, exts='ws', routes=mount_routes, highlightjs=True)
    # app.add_middleware(SessionSyncMiddleware)
    rt = app.route
    return app, rt

app, rt = create_app()

In [ ]:
MountPaths.gradio_url.value

'/obsidian_gradio'

In [ ]:
#| export
@rt
def index(request:Request):
    comp = homepage(config)
    return ifhtmx(request, comp)

In [ ]:
#| export
@rt
def edit(request:Request, session):
    obsidian_url = session['obsidian_url']
    resp = Response(status_code=200)
    # print(url)
    resp.headers["HX-Redirect"] = obsidian_url
    return resp
    

In [ ]:
#| export
def backlinkscomp(bls):
    if bls:
        print(bls)
        Lis = [Li(A(src.title, href=src.url, hx_push_url=src.url, hx_boost=True, id=f"backlink-{uuid.uuid4()}"
                )) for src in bls]
        # Lis = [Li(f"Checking {bls}")]
        return Ul(
            *Lis,
            id='backlink_list', 
            cls='menu bg-base-200 text-base-content min-h-full w-80 p-4',
            hx_swap_oob='outerHTML',
        )

         
    else:
        return Ul(Li("No Backlink referring to this document or not supported"), 
            id='backlink_list', 
            cls='menu bg-base-200 text-base-content min-h-full w-80 p-4',
            hx_swap_oob='outerHTML')

In [ ]:
#| export
def LinkCard(src):
    card = Card(H4(src.title),
                cls="p-4 flex flex-col items-center text-center",
                role="button",
                tabindex="0",
                style="cursor: pointer;"
            )
    return A(card, href=src.url, cls="block hover:shadow-lg transition duration-300")


In [ ]:
#| export
def LinksComp(links):
    if links: cards = [LinkCard(src) for src in links]
    else:cards = [Card(H4("No links found"))]
    return Grid(*cards, lg=3)
    

In [ ]:
#| export
@rt
def tags(request:Request, name:str):
    if config.ENABLE_DB_MODE: 
        store = DBStore(config)
        tls = store.tlinks(name)
        # print(tls, name)
        comp = Div(
            H2(f"Taglist for {name}"), 
            LinksComp(tls)
        )
        return ifhtmx(request,comp)
    else: return ifhtmx(request, Div("Taglist only supported in DBMODE"))

In [ ]:
#| export
@rt
def getbacklinks(request: Request,lockey:str):
    store = DBStore(config) if config.ENABLE_DB_MODE else FileStore(config)
    bls = store.backlinks(lockey) 
    return backlinkscomp(bls)

In [ ]:
# getbacklinks.to(lockey='test')

In [ ]:
#| export
@rt
def open(request: Request, session, file: str = "", title: Optional[str]=None):
    # VIDEO_EXTS = (".mp4", ".webm", ".ogg", ".mov", ".mkv")
    # AUDIO_EXTS = (".mp3", ".wav", ".ogg", ".m4a", ".flac")
    # IMAGE_EXTS = (".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp")
    # TEXTLIKE_EXTS = (".md", ".qmd", ".canvas", ".base")
    
    # Simple check: if it's HTMX, return HTML; otherwise return raw file
    is_htmx = request.headers.get('HX-Request')
    
    store = DBStore(config) if config.ENABLE_DB_MODE else FileStore(config)
    out = store.query_file(file)
    if out['content'] is None: return Div(f"Path not found: {urllib.parse.unquote(file)}")

    # If it's a directory → list contents
    if out['is_folder']:
        comp = Div(
            H2(f"Listing for {out['title']}"), 
            LinksComp(out['content'])
        )
        return ifhtmx(request, comp)
        
    elif out['response_type'] == ResponseTypes.html:
        html = out['content']
        session['obsidian_url'] = out['obsidian_url']
        session['lockey'] = out['lockey']
        print(out['backlinks'])
        return ifhtmx(
            request,
            Fragment(
                Container(
                    H1(out['title'], cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                    Card(NotStr(apply_classes(html))),
                    Hidden(id='current-lockey', value=out['lockey'], hx_swap_oob='true', name='current-lockey'),
                ), 
                Div(id="backlink_trigger",
                    hx_get=MountPaths.getbacklinks.to(lockey=out['lockey']),
                    hx_target="#backlink_list",
                    hx_swap=None,
                    hx_trigger="load",
                    style="display:none")
        # backlinkscomp(out['backlinks'])
            )

        )
    elif out['response_type'] == ResponseTypes.file:
        file = out['content']
        return FileResponse(file)
    elif out['response_type'] == ResponseTypes.blob:
        blob_data = out['content']
        if not isinstance(blob_data, (bytes, bytearray)):
            blob_data = bytes(blob_data)

        mime_type = guess_mime(f"file.{out['extension']}") or "application/octet-stream"
        extension = out['extension'].lower()
        
        # If not HTMX, return raw file
        if not is_htmx:
            return StreamingResponse(
                BytesIO(blob_data),
                media_type=mime_type,
                headers={
                    "Content-Disposition": f'inline; filename="{(title or "file")}.{extension.lstrip(".")}"',
                    "Cache-Control": "no-store",
                }
            )
        
        # For HTMX requests, embed as base64 data URLs
        import base64
        b64_data = base64.b64encode(blob_data).decode('utf-8')
        data_url = f"data:{mime_type};base64,{b64_data}"
        
        # Check file type and return appropriate HTML
        if extension in ExtensionTypes.IMAGE_EXTS:
            return ifhtmx(
                request,
                Container(
                    H1(out['title'] or title or "Image", cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                    Card(
                        Img(src=data_url, alt=out['title'] or title or "Image", 
                            style="max-width: 100%; height: auto;",
                            cls="uk-responsive-width")
                    )
                )
            )
        elif extension in ExtensionTypes.VIDEO_EXTS:
            return ifhtmx(
                request,
                Container(
                    H1(out['title'] or title or "Video", cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                    Card(
                        Video(
                            src=data_url,
                            controls=True,
                            style="max-width: 100%; height: auto;"
                        )
                    )
                )
            )
        elif extension in ExtensionTypes.AUDIO_EXTS:
            return ifhtmx(
                request,
                Container(
                    H1(out['title'] or title or "Audio", cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                    Card(
                        Audio(
                            src=data_url,
                            controls=True
                        )
                    )
                )
            )
        else:
            # For other file types, return as download/stream
            return StreamingResponse(
                BytesIO(blob_data),
                media_type=mime_type,
                headers={
                    "Content-Disposition": f'inline; filename="{(title or "file")}.{extension.lstrip(".")}"',
                    "Cache-Control": "no-store",
                }
            )
    else:
        return ifhtmx(request, Div("No Response"))

In [ ]:
#| export
def iter_file(path, chunk_size=8192):
    with open(path, "rb") as f:
        while chunk := f.read(chunk_size):
            yield chunk

In [ ]:
fname = 'Test.m4a'
guess_mime(fname)
# mime = mimetypes.guess_type(fname); mime[0]
# FileResponse?

'audio/mp4'

In [ ]:
#| export
def CollapsibleBlocks(title, comp):
    return Div(
            Input(type='checkbox'),
            Div(title, cls='collapse-title font-semibold'),
            Div(comp, cls='collapse-content text-sm'),
            cls='collapse collapse-plus border'
        )

In [ ]:
to_xml(CollapsibleBlocks('T', 'c'))

'<div class="collapse collapse-plus border">\n  <input type="checkbox" class="uk-input ">\n  <div class="collapse-title font-semibold">T</div>\n  <div class="collapse-content text-sm">c</div>\n</div>\n'

In [ ]:
#| export

@rt
def embed(request: Request, file: str = "", ext: Optional[str] = None, title: Optional[str] = None):

    store = DBStore(config) if config.ENABLE_DB_MODE else FileStore(config)
    out = store.query_file(file)
    if out["content"] is None: return Div(f"Path not found: {urllib.parse.unquote(file)}")
    if out["is_folder"]:
        comp = Div(
            H2(f"Listing for {out['title']}"), 
            LinksComp(out['content'])
        )
        return ifhtmx(request, comp)
            # items = []
            # for p in out["content"]:
            #     name = p["fname"]
            #     href = p["url"]
            #     items.append(Li(A(name, href=href)))
            # return ifhtmx(request, Div(f"Listing for {out['title']}:", Ul(*items)))
    if ext in ExtensionTypes.IMAGE_EXTS:
            headers = {"Content-Type": f"image/{ext.lstrip('.')}"}
            if title: headers["X-Image-Title"] = title
            alt_text = pathlib.Path(file).stem
            headers["X-Image-Alt"] = alt_text
            if out["response_type"] == ResponseTypes.file: return FileResponse(out["content"], headers=headers)
            elif out["response_type"] == ResponseTypes.blob: return StreamingResponse(BytesIO(out["content"]), headers=headers)

    if ext in ExtensionTypes.VIDEO_EXTS:
            mime = guess_mime(file) or f"video/{ext.lstrip('.')}"
            headers = {"Accept-Ranges": "bytes"}
            if out["response_type"] == ResponseTypes.file: return FileResponse(out["content"], media_type=mime, headers=headers, filename=title or file)
            elif out["response_type"] == ResponseTypes.blob: return StreamingResponse(BytesIO(out["content"]), media_type=mime, headers=headers)

        # --- Audio files ---
    if ext in ExtensionTypes.AUDIO_EXTS:
        mime = guess_mime(file) or f"audio/{ext.lstrip('.')}"
        headers = {"Accept-Ranges": "bytes"}
        if out["response_type"] == ResponseTypes.file: return FileResponse(out["content"], media_type=mime, headers=headers, filename=title or file)
        elif out["response_type"] == ResponseTypes.blob: return StreamingResponse(BytesIO(out["content"]), media_type=mime, headers=headers)

    if out["response_type"] == ResponseTypes.html:
        return CollapsibleBlocks(
            Div(out['title']),
            NotStr(apply_classes(out["content"])),
        )

    return ifhtmx(request, Div("Unsupported file type"))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()